In [2]:
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import statsmodels.api as sm
from statsmodels.tsa.arima_model import ARIMA, ARIMAResults
from statsmodels.tsa.arima_process import ArmaProcess
from statsmodels.stats.diagnostic import acorr_ljungbox
from scipy import signal
from scipy import stats

1. Please download the historical prices and dates of gold and silver from these 2 URLs (Jan 2019 till now):

 

•             https://www.investing.com/commodities/gold-historical-data

•             https://www.investing.com/commodities/silver-historical-data

 

and store them locally (in a file or database, as you see fit).

In [18]:
silver_df = pd.read_csv('./data/Silver Futures Historical Data.csv')
gold_df = pd.read_csv('./data/Gold Futures Historical Data.csv')

In [19]:
silver_df['Date']= pd.to_datetime(silver_df['Date'])
gold_df['Date']= pd.to_datetime(gold_df['Date'])

In [39]:
silver_df.head(20)

,Date,Price,Open,High,Low,Vol.,Change %
0,2019-06-11,14.738,14.695,14.783,14.633,-,0.28%
1,2019-06-10,14.697,14.973,14.998,14.627,-,-1.84%
2,2019-06-09,14.973,15.008,15.008,14.953,-,-0.21%
3,2019-06-07,15.005,14.852,15.148,14.842,-,1.00%
4,2019-06-06,14.857,14.793,14.988,14.732,-,0.43%
5,2019-06-05,14.793,14.812,15.037,14.758,-,-0.09%
6,2019-06-04,14.807,14.768,14.848,14.648,-,0.26%
7,2019-06-03,14.768,14.602,14.822,14.568,-,1.14%
8,2019-06-02,14.602,14.555,14.602,14.555,-,0.32%
9,2019-05-31,14.555,14.510,14.668,14.457,-,0.32%


In [21]:
gold_df.head()

,Date,Price,Open,High,Low,Vol.,Change %
0,2019-06-11,"1,331.55","1,333.25","1,333.75","1,323.65",-,0.52%
1,2019-06-10,"1,324.70","1,335.80","1,336.50","1,324.10",0.27K,-1.23%
2,2019-06-07,"1,341.20","1,333.10","1,347.70","1,329.80",0.24K,0.27%
3,2019-06-06,"1,337.60","1,330.80","1,338.70","1,326.20",0.16K,0.70%
4,2019-06-05,"1,328.30","1,328.90","1,343.30","1,326.30",0.62K,0.37%


In [40]:
def preciousMetal(start_date,end_date,commodity_type):
    df=commodity_type+'_df'
    print (df,start_date,end_date)
    df = df[(df['Date'] >= start_date) & (df['Date'] <= end_date)]
    print (df)
    

In [38]:
silver_df[(silver_df['Date']>='2019-06-01')&(silver_df['Date']<='2019-06-05')]

,Date,Price,Open,High,Low,Vol.,Change %
5,2019-06-05,14.793,14.812,15.037,14.758,-,-0.09%
6,2019-06-04,14.807,14.768,14.848,14.648,-,0.26%
7,2019-06-03,14.768,14.602,14.822,14.568,-,1.14%
8,2019-06-02,14.602,14.555,14.602,14.555,-,0.32%


In [42]:
preciousMetal('2019-05-10','2019-05-22','silver')

silver_df 2019-05-10 2019-05-22


TypeError: string indices must be integers

2. Please then write a program that takes the following 3 command line arguments:

•             Start date (in the format 2017-05-10)

•             End date (in the format 2017-05-22)

•             Commodity type (either "gold" or silver”)

Then first describe the statistical properties of the data. Next, tell whether the data are stationary. Lastly, provide additional time series properties of interest, if any.

3. Write a program to help you decide how useful the previous gold or silver prices are as predictors for their future prices.

Also determine if past prices are useful in predicting the direction of change (increase or decrease). There is no need to provide future predictions, and you may use whatever tests you deem useful.

Please feel free to use whatever language you think is best suited to the task and/or familiar to you.

Please also feel free to consult references (e.g. hard to remember libraries, function names, call signatures, etc.) as necessary, and don’t be shy about asking for help if you get stuck.